### List of All Senses
{'',
 'Comparison.Concession+SpeechAct.Arg2-as-denier+SpeechAct',
 'Comparison.Concession.Arg1-as-denier',
 'Comparison.Concession.Arg2-as-denier',
 'Comparison.Contrast',
 'Comparison.Similarity',
 'Contingency.Cause+Belief.Reason+Belief',
 'Contingency.Cause+Belief.Result+Belief',
 'Contingency.Cause+SpeechAct.Reason+SpeechAct',
 'Contingency.Cause+SpeechAct.Result+SpeechAct',
 'Contingency.Cause.Reason',
 'Contingency.Cause.Result',
 'Contingency.Condition+SpeechAct',
 'Contingency.Condition.Arg1-as-cond',
 'Contingency.Condition.Arg2-as-cond',
 'Contingency.Negative-cause.NegResult',
 'Contingency.Negative-condition.Arg1-as-negCond',
 'Contingency.Negative-condition.Arg2-as-negCond',
 'Contingency.Purpose.Arg1-as-goal',
 'Contingency.Purpose.Arg2-as-goal',
 'Expansion.Conjunction',
 'Expansion.Disjunction',
 'Expansion.Equivalence',
 'Expansion.Exception.Arg1-as-excpt',
 'Expansion.Exception.Arg2-as-excpt',
 'Expansion.Instantiation.Arg1-as-instance',
 'Expansion.Instantiation.Arg2-as-instance',
 'Expansion.Level-of-detail.Arg1-as-detail',
 'Expansion.Level-of-detail.Arg2-as-detail',
 'Expansion.Manner.Arg1-as-manner',
 'Expansion.Manner.Arg2-as-manner',
 'Expansion.Substitution.Arg1-as-subst',
 'Expansion.Substitution.Arg2-as-subst',
 'Temporal.Asynchronous.Precedence',
 'Temporal.Asynchronous.Succession',
 'Temporal.Synchronous'}
 
CAUSAL SENSES ARE:

In [1]:
# split arg1arg2 first
arg2IsCause = [
    'Contingency.Cause.Reason',
    'Contingency.Purpose.Arg2-as-goal',
    'Contingency.Condition.Arg2-as-cond',
    'Contingency.Negative-condition.Arg2-as-negCond'
]
arg1IsCause = [
    'Contingency.Cause.Result',
    'Contingency.Purpose.Arg1-as-goal',
    'Contingency.Condition.Arg1-as-cond',
    'Contingency.Negative-condition.Arg1-as-negCond',
    'Contingency.Negative-cause.NegResult'
]

causal_senses = arg2IsCause + arg1IsCause

### PDTB to Pickle Dict Format


In [2]:
import re
import numpy as np
import os
from tqdm import tqdm
from collections import defaultdict
import pandas as pd

In [3]:
raw_dir = r"D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\raw"
gold_dir = r"D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold"
all_filenames = [f for dp, dn, filenames in os.walk(raw_dir) for f in filenames]

In [4]:
def open_txt(filename):
    with open(filename, "rb") as f:
        content = f.readlines()
    content = [x.decode("UTF-8",errors='ignore') for x in content]
    return content


def parse_txt_file(fn, buffer = 10):
    
    document_ = open_txt(fn)
        
    sent2locid = {-1:0}
    document = ''
    sent_id = 0

    # alr split by lines
    sents = []
    for doc in document_:
        doc=re.sub('\n','',doc)
        document+=str(doc)+' '
        if doc!='':
            sents.append(doc)
    
    for sent in sents:
        if sent=='':
            sent2locid[sent_id-1]+=1
            continue
        start_from=max(0,sent2locid[sent_id-1]-buffer)
        sent2locid[sent_id]=re.search(re.escape(sent),document[start_from:]).end()+1+start_from
        sent_id+=1
    
    sent2locid[max(sent2locid.keys())+1]=len(document)

    return document[:-1], sent2locid



def convert_sents(_sents):
    _sents = list(set(_sents))
    if len(_sents)==0:
        raise ValueError('Argument does not have any sentence.')
    else:
        return [int(s) for s in _sents]


def find_sents_needed(sent2locid, search_min, search_max):
    sents_needed = []
    for k,v in sent2locid.items():
        if v<=search_min:
            # keep replacing as first item
            sents_needed = [k+1]
            continue
        if v>search_max:
            # sufficient found, exit
            break
        sents_needed.append(k+1)
    return sents_needed


def jump(sentid):
    if isinstance(sentid, (str, int)):
        return [int(sentid)]
    else:
        return sentid


def if_consecutive(l):
    return sorted(l) == list(range(min(l), max(l)+1))

        
def run(file):
    # Open Files
    filename = f"{file[4:6]}\\{file}"
    gold_path = os.path.join(gold_dir, filename)
    raw_path = os.path.join(raw_dir, filename)
    
    if not (os.path.isfile(gold_path) and os.path.isfile(raw_path)):
        print(f'Missing: {gold_path}')
        return pd.DataFrame()
    
    fn = raw_path
    document, sent2locid = parse_txt_file(raw_path, buffer = 10)
    rels = open_txt(gold_path)
    
    # Format Text
    sentid_to_text = {}
    for k,v in sent2locid.items():
        if k<0:
            continue
        else:
            sentid_to_text[k]=document[sent2locid[k-1]:v]
    
    # If not in dict, build in place
    def get_e_to_loc(es):
        if es not in eid_to_loc.keys():
            start, end = es.split('..')
            start = int(start)
            end = int(end)
            _sents = find_sents_needed(sent2locid, start, end)
            if len(_sents)>1:
                start_list, end_list = [start], []
                for s in _sents[:-1]:
                    brk = sent2locid[s]
                    end_list.append(brk)
                    start_list.append(brk)
                end_list.append(end)
                eid_to_loc[es] = (start_list, end_list, _sents)
            else:
                eid_to_loc[es] = ([int(start)], [int(end)], _sents)
        return eid_to_loc[es]

    # Multi-word build
    def get_e_to_loc_multi(es):
        if es not in eid_to_loc.keys():
            es_list = [r for r in es.split(';') if r!='']

            _min, _max = np.inf, 0
            _sents = []
            consequetive_check=True
            prev_endloc = -1
            for e in es_list:
                start, end, sentid = get_e_to_loc(e)
                if len(sentid)>1: # do not join if across sentence
                    consequetive_check=False
                _min = min(min(start),_min)
                _max = max(max(end),_max)
                _sents.extend(sentid)
                if prev_endloc<0:
                    # init
                    prev_endloc=int(max(end))
                else:
                    if prev_endloc+1!=int(max(end)):
                        consequetive_check=False
                    prev_endloc=int(max(end))

            if consequetive_check:
                eid_to_loc[es] = ([_min], [_max], _sents)
            else:
                if len(es_list)>1:
                    start, end = [], []
                    for e in es_list:
                        start.extend(eid_to_loc[e][0])
                        end.extend(eid_to_loc[e][1])
                assert(len(start)==len(end))
                assert(len(start)==len(_sents))
                eid_to_loc[es] = (start, end, _sents)

        return eid_to_loc[es]
    
    # Format Annotations
    def get_text_from_sentid(sentid):
        if isinstance(sentid, (str)):
            splitted = sentid.split(';')
            text = ''.join([sentid_to_text[int(s)] for s in splitted])
            accounting = sent2locid[int(splitted[0])-1]
        else:
            text = sentid_to_text[sentid]
            accounting = sent2locid[sentid-1]
        return text, accounting

    def get_s_e_t_dict(h,t):
        s_head, e_head, sentid_head = get_e_to_loc_multi(h)
        s_tail, e_tail, sentid_tail = get_e_to_loc_multi(t)
        
#         print(eid_to_loc)
#         print(sentid_head, sentid_tail)

        s_locs, e_locs, tags = [], [], []
        sentid_to_pair_locs = defaultdict(list)

        for i, s in enumerate(sentid_head):
            sentid_to_pair_locs[s].append((s_head[i], e_head[i], '<ARG0>'))
        for i, s in enumerate(sentid_tail):
            sentid_to_pair_locs[s].append((s_tail[i], e_tail[i], '<ARG1>'))

        for k,v in sentid_to_pair_locs.items():
            sentid_to_pair_locs[k] = sorted(v)
        return sentid_to_pair_locs #sorted(zip(s_locs, e_locs, tags))
    
    eid_to_loc = {} # reset
    rel_to_es = {} # rel_to_es[lid]=(headid,tailid)
    causal_rels = []

    for relID, rel in enumerate(rels):
        rel = rel.split('|')
        get_e_to_loc_multi(rel[14])
        get_e_to_loc_multi(rel[20])
        
        if rel[8] in causal_senses:
            causal_rels.append(relID)
            if rel[8] in arg2IsCause:
                rel_to_es[relID] = (rel[20],rel[14])
            else:
                rel_to_es[relID] = (rel[14], rel[20])
        else:
            rel_to_es[relID] = (rel[14], rel[20])
        
    # Generate Examples
    # Retain rels that are in same sentence
    corpus = 'pdtb'
    eid_to_loc = {} # reset

    cols = ['corpus','doc_id','sent_id','eg_id','index','text','text_w_pairs','seq_label','pair_label','context','num_sents']
    data = []
    sentid_counter = defaultdict(int)

    for rel_id, (h, t) in rel_to_es.items():
        
        sentid_to_pair_locs = get_s_e_t_dict(h,t)
        sents_needed = list(sentid_to_pair_locs.keys())

        tmp_sents_text = {}

        sentid = ';'.join([str(s) for s in sents_needed])
        num_eg_for_this_sentid = sentid_counter[sentid]
        identifiers = [corpus,os.path.basename(fn),str(sentid),str(num_eg_for_this_sentid)]
        unique_index = '_'.join(identifiers)

        for _sentid, locs_to_edit in sentid_to_pair_locs.items():
            added_t = 0
            accounting = sent2locid[_sentid-1]
            text_w_pairs = sentid_to_text[_sentid]
            for s,e,t in locs_to_edit:
                s = s+added_t-accounting
                e = e+added_t-accounting
                text_w_pairs = text_w_pairs[:s]+t+text_w_pairs[s:e]+t[0]+'/'+t[1:]+text_w_pairs[e:]
                added_t+=13
            tmp_sents_text[_sentid] = text_w_pairs.strip()

        seq_label = pair_label = 1 if rel_id in causal_rels else 0 

        text_w_pairs = ' '.join(tmp_sents_text.values())
        text = ' '.join([sentid_to_text[s].strip() for s in sents_needed])
        num_sents = max(sents_needed)-min(sents_needed)+1
        if if_consecutive(sents_needed):
            context = ''
        else:
            context = ' '.join([sentid_to_text[s].strip() for s in range(max(sents_needed), max(sents_needed)+1)])

        data.append(
            identifiers+[
                unique_index,
                text.strip(),
                text_w_pairs.strip(),
                seq_label,
                pair_label,
                context,
                num_sents
            ]
        )
        sentid_counter[sentid]+=1

    data = pd.DataFrame(data, columns=cols)
    data['seq_label'] = data.groupby(['corpus','doc_id','sent_id'])['seq_label'].transform('max')
    
    return data

### ONE FILE TRIAL
file = all_filenames[20]
data = run(file)
data

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,pdtb,wsj_0021,2;3,0,pdtb_wsj_0021_2;3_0,The declaration by Economy Minister Nestor Rap...,<ARG0>The declaration by Economy Minister Nest...,0,0,,2
1,pdtb,wsj_0021,5;6,0,pdtb_wsj_0021_5;6_0,Mr. Rapanelli met in August with U.S. Assistan...,<ARG0>Mr. Rapanelli met in August with U.S. As...,0,0,,2
2,pdtb,wsj_0021,6,0,pdtb_wsj_0021_6_0,Argentine negotiator Carlos Carballo was in Wa...,<ARG1>Argentine negotiator Carlos Carballo was...,1,1,,1
3,pdtb,wsj_0021,7;8,0,pdtb_wsj_0021_7;8_0,Mr. Rapanelli recently has said the government...,<ARG0>Mr. Rapanelli recently has said the gove...,0,0,,2


In [5]:
data = pd.DataFrame()
for counter, file in enumerate(all_filenames):

    df = run(file)
    data = pd.concat([data, df])

data

Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0002
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0006
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0017
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0028
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0030
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0032
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0042
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0053
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0055
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0056
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\00\wsj_0058
Missing: D:\08 Thesis\09 Datasets\07. PennT

Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0835
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0836
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0837
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0838
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0839
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0840
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0841
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0842
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0843
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0844
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\08\wsj_0845
Missing: D:\08 Thesis\09 Datasets\07. PennT

Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\14\wsj_1420
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\14\wsj_1425
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\14\wsj_1427
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\14\wsj_1444
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\14\wsj_1482
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\15\wsj_1506_e
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\15\wsj_1559
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\15\wsj_1576
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\15\wsj_1579
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\16\wsj_1608
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\16\wsj_1625
Missing: D:\08 Thesis\09 Datasets\07. Pen

Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\23\wsj_2318
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\23\wsj_2349
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\23\wsj_2385
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\23\wsj_2388
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\23\wsj_2389
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\23\wsj_2390
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\24\wsj_2455
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\24\wsj_2456
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\24\wsj_2457
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\24\wsj_2458
Missing: D:\08 Thesis\09 Datasets\07. PennTreeBank V3.0\PDTB-3.0\data\gold\24\wsj_2459
Missing: D:\08 Thesis\09 Datasets\07. PennT

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,pdtb,wsj_0001,1;2,0,pdtb_wsj_0001_1;2_0,"Pierre Vinken, 61 years old, will join the boa...","<ARG0>Pierre Vinken, 61 years old, will join t...",0,0,,2
0,pdtb,wsj_0003,2,0,pdtb_wsj_0003_2_0,"The asbestos fiber, crocidolite, is unusually ...","<ARG0>The asbestos fiber, crocidolite, is unus...",0,0,,1
1,pdtb,wsj_0003,2,1,pdtb_wsj_0003_2_1,"The asbestos fiber, crocidolite, is unusually ...","<ARG0>The asbestos fiber, crocidolite, is unus...",0,0,,1
2,pdtb,wsj_0003,2;3,0,pdtb_wsj_0003_2;3_0,"The asbestos fiber, crocidolite, is unusually ...","<ARG0>The asbestos fiber, crocidolite, is unus...",0,0,,2
3,pdtb,wsj_0003,4,0,pdtb_wsj_0003_4_0,Although preliminary findings were reported mo...,Although <ARG1>preliminary findings were repor...,0,0,,1
...,...,...,...,...,...,...,...,...,...,...,...
49,pdtb,wsj_2454,31,1,pdtb_wsj_2454_31_1,"At last night's rally, they called on their fo...","At last night's rally, they called on their fo...",1,1,,1
50,pdtb,wsj_2454,31,2,pdtb_wsj_2454_31_2,"At last night's rally, they called on their fo...","At last night's rally, they called on their fo...",1,1,,1
51,pdtb,wsj_2454,31,3,pdtb_wsj_2454_31_3,"At last night's rally, they called on their fo...","At last night's rally, they called on their fo...",1,0,,1
52,pdtb,wsj_2454,31,4,pdtb_wsj_2454_31_4,"At last night's rally, they called on their fo...","At last night's rally, they called on their fo...",1,0,,1


In [6]:
list(data['text'])[0]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29. Mr. Vinken is chairman of Elsevier N.V., the Dutch publishing group.'

In [7]:
from collections import Counter

print('All Examples')
print('Seq Level:', Counter(data['seq_label'])) # if sentence level causality exists
print('Pair Level:',Counter(data['pair_label'])) # if ARG0-ARG1 pair level causality exists
print('Seq Level (Unique):',Counter(data.drop_duplicates(subset=['corpus','doc_id','sent_id'])['seq_label']))

print('\nSingle Sentence Examples')
print('Seq Level:', Counter(data.loc[data['num_sents']==1,'seq_label'])) # if sentence level causality exists
print('Pair Level:', Counter(data.loc[data['num_sents']==1,'pair_label'])) # if ARG0-ARG1 pair level causality exists
print('Seq Level (Unique):',Counter(data.loc[data['num_sents']==1].drop_duplicates(subset=['corpus','doc_id','sent_id'])['seq_label']))

All Examples
Seq Level: Counter({0: 36034, 1: 17642})
Pair Level: Counter({0: 41420, 1: 12256})
Seq Level (Unique): Counter({0: 31786, 1: 11064})

Single Sentence Examples
Seq Level: Counter({0: 14272, 1: 13816})
Pair Level: Counter({0: 19597, 1: 8491})
Seq Level (Unique): Counter({0: 10355, 1: 7304})


In [8]:
data.to_csv('cleaned/pdtb.csv', index=False,encoding='utf-8-sig')